In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pyobjc

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [9]:
import streamlit as st 
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import objc
from Foundation import NSSpeechSynthesizer, NSAutoreleasePool

In [10]:
def get_pdf_text(pdf_path):
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

#지정된 조건에 따라 주어진 텍스트를 더 작은 덩어리로 분할
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators="\\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

#주어진 텍스트 청크에 대한 임베딩을 생성하고 FAISS를 사용하여 벡터 저장소를 생성
def get_vectorstore(text_chunks):
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [16]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

#주어진 벡터 저장소로 대화 체인을 초기화
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_message=True)  #ConversationBufferWindowMemory에 이전 대화 저장
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo-16k-0613'),
        retriever=vectorstore.as_retriever(),
        get_chat_history=lambda h: h,
        memory=memory
    ) #ConversationalRetrievalChain을 통해 langchain 챗봇에 쿼리 전송
    return conversation_chain

In [17]:
# PDF 파일 경로 설정
pdf_path = "data/The_Adventures_of_Tom_Sawyer.pdf"
# PDF 텍스트 가져오기
raw_text = get_pdf_text(pdf_path)
# 텍스트에서 청크 검색
text_chunks = get_text_chunks(raw_text)
# PDF 텍스트 저장을 위해 FAISS 벡터 저장소 만들기
vectorstore = get_vectorstore(text_chunks)
# 대화 체인 만들기
m = get_conversation_chain(vectorstore)

In [ ]:
import speech_recognition as sr
import pyttsx3

# 음성 입력 (STT)
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("질문을 해주세요.")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio, language="ko-KR")
        print("사용자:", question)
        return question
    except sr.UnknownValueError:
        print("음성을 이해할 수 없습니다.")
        return ""
    except sr.RequestError:
        print("음성 서비스에 접근할 수 없습니다.")
        return ""

# 음성 출력 (TTS)
def speak_response(response):
    pool = NSAutoreleasePool.alloc().init()
    speaker = NSSpeechSynthesizer.alloc().initWithVoice_(None)
    speaker.startSpeakingString_(response)

# 질문 처리 및 답변 생성
def process_question(question):
    # 질문에 대한 답변 가져오기
    response = m({"question": question})
    if 'answer' in response:
        answer = response['answer']
    else:
        print("올바른 응답 키를 찾을 수 없습니다.")
    return answer

if __name__ == "__main__":
    while True:
        question = recognize_speech()
        if question=="멈춰":
            break;
        if question:
            answer = process_question(question)
            print(answer)
            speak_response(answer)


질문을 해주세요.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


사용자: 줄거리 설명해 줘
